<a href="https://colab.research.google.com/github/Aayushkumr/Music-Recommender-System/blob/main/Aayush_Kumar's_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Music Recommendation System using Spotify Dataset & Spotify APIs

### 4th Semester Project Work

##### Aayush Kumar
##### Roll No.- 12112126
##### CS-B6

In [ ]:
import os
import warnings
import numpy as np
import pandas as pd

import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import euclidean_distances
from scipy.spatial.distance import cdist

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv("spotify_data/data.csv")
genre_data = pd.read_csv('spotify_data/data_w_genres.csv')
year_data = pd.read_csv('spotify_data//data_by_year.csv')
artist_data = pd.read_csv('spotify_data//data_by_artist.csv')

In [ ]:
data.head(5)

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.982,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.732,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.160,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.961,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.101,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.967,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.381,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.957,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.229,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665


In [ ]:
#grouping years into decades as a new column
def get_decade(year):
    period_start = int(year/10) * 10
    decade = '{}s'.format(period_start)
    return decade
data['decade'] = data['year'].apply(get_decade)

In [ ]:
data['decade']

0        1920s
1        1920s
2        1920s
3        1920s
4        1920s
         ...  
78456    1950s
78457    1950s
78458    1950s
78459    1950s
78460    1950s
Name: decade, Length: 78461, dtype: object

## Scaling & Clustering genre_data

In [ ]:
useful_cols = ['acousticness', 'danceability', 'energy', 'instrumentalness','liveness', 'loudness', 'speechiness', 'tempo', 'valence','duration_ms','key','mode']

In [ ]:
scaled_genre_data = StandardScaler().fit(genre_data[useful_cols])

In [ ]:
genre_data.head(3)

,genres,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count
0,['show tunes'],"""Cats"" 1981 Original London Cast",0.590111,0.467222,250318.555556,0.394003,0.011400,0.290833,-14.448000,0.210389,117.518111,0.389500,38.333333,5,1,9
1,[],"""Cats"" 1983 Broadway Cast",0.862538,0.441731,287280.000000,0.406808,0.081158,0.315215,-10.690000,0.176212,103.044154,0.268865,30.576923,5,1,26
2,[],"""Fiddler On The Roof” Motion Picture Chorus",0.856571,0.348286,328920.000000,0.286571,0.024593,0.325786,-15.230714,0.118514,77.375857,0.354857,34.857143,0,1,7


In [ ]:
scaled_genre_data.mean_ , scaled_genre_data.var_

(array([ 4.98372653e-01,  5.46490069e-01,  4.97488151e-01,  1.74756077e-01,
         2.02440841e-01, -1.11404984e+01,  9.40143659e-02,  1.15844830e+02,
         5.12722840e-01,  2.38877995e+05,  5.41290098e+00,  7.59170153e-01]),
 array([1.37350304e-01, 3.11419583e-02, 6.49641344e-02, 8.90430979e-02,
        1.98475382e-02, 3.33119304e+01, 1.25404485e-02, 6.25169939e+02,
        5.97394658e-02, 1.46724112e+10, 1.21138224e+01, 1.82830832e-01]))

In [ ]:
scaled_genre_data = scaled_genre_data.transform(genre_data[useful_cols])

In [ ]:
scaled_genre_data

array([[ 0.24753501, -0.44918382, -0.40601288, ...,  0.09444882,
        -0.11863294,  0.56322976],
       [ 0.98261718, -0.59363517, -0.35577619, ...,  0.39958816,
        -0.11863294,  0.56322976],
       [ 0.96651653, -1.12315639, -0.82751178, ...,  0.74335182,
        -1.55521158,  0.56322976],
       ...,
       [ 0.77462407,  0.13785911, -1.36961429, ..., -0.53079676,
        -0.11863294,  0.56322976],
       [-0.31670295, -1.09644214,  0.73960838, ...,  0.63801022,
         1.60526142,  0.56322976],
       [ 0.18787329, -0.56377624, -1.1083155 , ..., -0.00982414,
         1.31794569,  0.56322976]])

#Using K means clustering algorithm to visualize various clusters in the given genre data


In [ ]:
import sklearn.cluster as cluster

Kmeans = cluster.KMeans(n_clusters=10, init="k-means++")
Kmeans = Kmeans.fit_predict(scaled_genre_data)


In [ ]:
Kmeans

array([1, 7, 7, ..., 1, 5, 7], dtype=int32)

In [ ]:
genre_data['Cluster'] = Kmeans
X = genre_data.select_dtypes(np.number)

In [ ]:
genre_data.head(5)

,genres,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count,Cluster
0,['show tunes'],"""Cats"" 1981 Original London Cast",0.590111,0.467222,250318.555556,0.394003,0.011400,0.290833,-14.448000,0.210389,117.518111,0.389500,38.333333,5,1,9,1
1,[],"""Cats"" 1983 Broadway Cast",0.862538,0.441731,287280.000000,0.406808,0.081158,0.315215,-10.690000,0.176212,103.044154,0.268865,30.576923,5,1,26,7
2,[],"""Fiddler On The Roof” Motion Picture Chorus",0.856571,0.348286,328920.000000,0.286571,0.024593,0.325786,-15.230714,0.118514,77.375857,0.354857,34.857143,0,1,7,7
3,[],"""Fiddler On The Roof” Motion Picture Orchestra",0.884926,0.425074,262890.962963,0.245770,0.073587,0.275481,-15.639370,0.123200,88.667630,0.372030,34.851852,0,1,27,7
4,[],"""Joseph And The Amazing Technicolor Dreamcoat""...",0.510714,0.467143,270436.142857,0.488286,0.009400,0.195000,-10.236714,0.098543,122.835857,0.482286,43.000000,5,1,7,1


# Visualizing the Clusters in genre data with t-SNE

## It is basically converting this multi-dimentional data into 2 dimensions x & y to project it (or visualize it) easily in graphical format

In [ ]:
from sklearn.manifold import TSNE

tsne_pipeline = Pipeline([('scaler', StandardScaler()), ('tsne', TSNE(n_components=2, verbose=1))])

genre_embedding = tsne_pipeline.fit_transform(X)
projection = pd.DataFrame(columns=['x', 'y'], data=genre_embedding)
projection['genres'] = genre_data['genres']
projection['cluster'] = genre_data['Cluster']

fig = px.scatter(projection, x='x', y='y', color='cluster', hover_data=['x', 'y', 'genres'])
fig.show()

## Scaling & Clustering Song Data

Using Elbow method to find no. of clusters required

Applying K-means clustering with calculated no of clusters

In [ ]:
song_cluster_pipeline = Pipeline([('scaler', StandardScaler()),('kmeans', KMeans(n_clusters=20,verbose=False))],
                                 verbose=False)

# here 'data' is song_data
X = data.select_dtypes(np.number)

number_cols = list(X.columns)
song_cluster_pipeline.fit(X)
song_cluster_labels = song_cluster_pipeline.predict(X)
data['cluster_label'] = song_cluster_labels

# Visualizing the Clusters with PCA
#### Principal Component Analysis or PCA is a dimensionality-reduction method that is often used to reduce the dimensionality of large data sets.

#### PCA Steps:


In [ ]:
from sklearn.decomposition import PCA

pca_pipeline = Pipeline([('scaler', StandardScaler()), ('PCA', PCA(n_components=2))])
song_embedding = pca_pipeline.fit_transform(X)
projection = pd.DataFrame(columns=['x', 'y'], data=song_embedding)
projection['title'] = data['name']
projection['cluster'] = data['cluster_label']

fig = px.scatter(
    projection, x='x', y='y', color='cluster', hover_data=['x', 'y', 'title'])
fig.show()

## Using Spotify APIs (using spotipy: python library )

In [ ]:
pip install spotipy

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
c_id = 'd93d61efd33f464fa828986f5be14f11'
c_secret = '216b06c55e93409b95fac81c14dd2f52'
client_credentials_manager = SpotifyClientCredentials(client_id=c_id, client_secret=c_secret)
sp = spotipy.Spotify(client_credentials_manager= client_credentials_manager)

In [ ]:
song = sp.search(q='track:perfect')

In [ ]:
for i in song['tracks']['items']:
 print(i['name'])

Perfect
She Looks So Perfect
The Perfect Girl
Perfect
Perfect
Perfect
Perfect
Perfect
FUCKING YOUNG / PERFECT (feat. Charlie Wilson, Chaz Bundick, Sydney Bennett & Kali Uchis)
Nobody's Perfect


In [ ]:
len(song['tracks']['items'])

10

In [ ]:
from collections import defaultdict

def find_song(name):
    song_data = defaultdict()
    results = sp.search(q= 'track: {} '.format(name), limit=1)
   # print(results)
    if results['tracks']['items'] == []:
        print('song not found')
    if results['tracks']['items'] == []:
        return None

    results = results['tracks']['items'][0]

    track_id = results['id']
    audio_features = sp.audio_features(track_id)[0]

   # print(audio_features)

    song_data['name'] = [results['name']]
    song_data['year'] = [int(results['album']['release_date'][0:4])]
    song_data['explicit'] = [int(results['explicit'])]
    song_data['duration_ms'] = [results['duration_ms']]
    song_data['popularity'] = [results['popularity']]

    for key, value in audio_features.items():
        song_data[key] = value

    return pd.DataFrame(song_data)

In [ ]:
mysong = find_song('anime')
mysong

In [ ]:
from collections import defaultdict
from sklearn.metrics import euclidean_distances
from scipy.spatial.distance import cdist
import difflib

number_cols = ['valence','year', 'acousticness', 'danceability', 'duration_ms', 'energy', 'explicit',
 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'popularity', 'speechiness', 'tempo']


def get_song_data(song, spotify_data):

#      try:
#          song_data = spotify_data[(spotify_data['name'] == song['name'])
#                                  & (spotify_data['year'] == song['year'])].iloc[0]
#          return song_data

#      except IndexError:
        return find_song(song['name'])

#### In this function we are taking the mean of all the song vectors to get a single mean vector which will be used for recommendation  



In [ ]:
def get_mean_vector(song_list, spotify_data):

    song_vectors = []

    for song in song_list:
        song_data = get_song_data(song, spotify_data)
        if song_data is None:
            print('Warning: {} does not exist in Spotify or in database'.format(song['name']))
            continue
        song_vector = song_data[number_cols].values
        song_vectors.append(song_vector)

    song_matrix = np.array(list(song_vectors))
    return np.mean(song_matrix, axis=0)


def flatten_dict_list(dict_list):

    flattened_dict = defaultdict()
   # print(dict_list)
    for key in dict_list[0].keys():
        flattened_dict[key] = []

    for dictionary in dict_list:
        # print(dictionary)
        for key, value in dictionary.items():
            flattened_dict[key].append(value)

    return flattened_dict


def recommend_songs( song_list, spotify_data, n_songs=10):

    metadata_cols = ['name','year', 'artists']
   # print(song_list)
    song_dict = flatten_dict_list(song_list)

    song_center = get_mean_vector(song_list, spotify_data)
    scaler = song_cluster_pipeline.steps[0][1]
   # print(song_cluster_pipeline)
    scaled_data = scaler.transform(spotify_data[useful_cols])
    scaled_song_center = scaler.transform(song_center.reshape(1, -1))
    #we are calculating euledian distances of this mean vector from each record within the song_data
    distances = cdist(scaled_song_center, scaled_data, 'cosine')
    # sorting the distnces to get the nearest songs from dataset
    index = list(np.argsort(distances)[:, :n_songs][0])

    # returning the nearest songs
    rec_songs = spotify_data.iloc[index]
    rec_songs = rec_songs[~rec_songs['name'].isin(song_dict['name'])]
    return rec_songs['name'].to_dict()

In [ ]:
get_mean_vector([{'name': 'hawayein'},{'name': 'perfect'}], data)

array([[ 3.105000e-01,  2.017000e+03,  4.795000e-01,  6.025000e-01,
         2.766995e+05,  5.245000e-01,  0.000000e+00,  2.555000e-04,
         5.000000e+00,  1.165000e-01, -5.944000e+00,  1.000000e+00,
         7.750000e+01,  2.575000e-02,  1.105055e+02]])

In [ ]:
recommend_songs([{'name': 'hawayein','year' :2001},{'name': 'perfect', 'year' : 2009}], data, 5)

{17027: 'Bleeding Love',
 17843: 'Country Boy (feat. George Jones & Charlie Daniels)',
 56725: 'The Village',
 56834: "I Don't Want You Back",
 57021: 'Some Things Never Change'}